In [1]:
from pprint import pprint
import reframed
pprint(dir(reframed))

['CAFBA',
 'CBModel',
 'CBReaction',
 'Community',
 'Compartment',
 'Environment',
 'FBA',
 'FBrAtio',
 'FVA',
 'GIMME',
 'GPRAssociation',
 'Gene',
 'MOMA',
 'Metabolite',
 'Model',
 'ModelCache',
 'NET',
 'Protein',
 'ROOM',
 'Reaction',
 'ReactionType',
 'SteadyCom',
 'SteadyComVA',
 'TFA',
 'TVA',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'blocked_reactions',
 'cobra',
 'community',
 'core',
 'eFlux',
 'escher_maps',
 'essential_genes',
 'essential_reactions',
 'external',
 'fluxes2escher',
 'from_cobrapy',
 'gene_knockout',
 'gpr_transform',
 'hard_knockout',
 'io',
 'lMOMA',
 'llFBA',
 'load_cbmodel',
 'make_irreversible',
 'marge',
 'minimal_medium',
 'pFBA',
 'plot_flux_envelope',
 'reaction_knockout',
 'save_cbmodel',
 'set_default_solver',
 'simplify',
 'solver_instance',
 'solvers',
 'to_cobrapy']


In [1]:
from reframed import *
from math import inf

In [5]:
def createIndModel(model, modelId):
    ind_model = model.copy()
    ind_model.id = modelId
    
    for comp in model.compartments:
        if 'e_' in comp:
            ind_model.compartments[comp].external = True

        #if (comp != 'u') and (modelId not in comp):
        if modelId not in comp:
            ind_model.remove_compartment(comp) # removes metabolites/reactions as well

    for rxn_id in ind_model.reactions:
        if '_u_' in rxn_id:
            rxn = ind_model.reactions[rxn_id]
            rxn.reaction_type = ReactionType.EXCHANGE
        
        if 'R_ATPM_' in rxn_id:
            ind_model.set_flux_bounds(rxn_id, 0, 0) # included when testing community model in test_reframed.py
    
    return ind_model

model = load_cbmodel('model9.xml',flavor='fbc2') # flavors: fbc2, cobra, bigg
# ba, bt, ec, ef, er, fp, kp, lc, st

model_ba = createIndModel(model,'Ba')
model_bt = createIndModel(model,'Bt')
model_ec = createIndModel(model,'Ec')
model_ef = createIndModel(model,'Ef')
model_er = createIndModel(model,'Er')
model_fp = createIndModel(model,'Fp')
model_kp = createIndModel(model,'Kp')
model_lc = createIndModel(model,'Lc')
model_st = createIndModel(model,'St')

community = Community('9member_model', [model_ba,model_bt,model_ec,model_ef,model_er,model_fp,model_kp,model_lc,model_st])

# constraints from SteadyCom paper
maxFiberUTBT = ['R_EX_amannan140_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_pect_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_pecticgal_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_arabinan101_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_arabinogal_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_homogal_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_inulin_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_kesto_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_rhamnogalurII_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_rhamnogalurI_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_kestopt_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_kestottr_LSQBKT_u_RSQBKT_tr_Bt']

maxFiberUTFP = ['R_IEX_pect_LSQBKT_u_RSQBKT_tr_Fp', 'comModel.reactions.R_IEX_inulin_LSQBKT_u_RSQBKT_tr_Fp', \
                'R_IEX_kesto_LSQBKT_u_RSQBKT_tr_Fp', 'comModel.reactions.R_IEX_kestopt_LSQBKT_u_RSQBKT_tr_Fp', \
                'R_IEX_kestottr_LSQBKT_u_RSQBKT_tr_Fp']

for rxn_id in community.merged_model.get_exchange_reactions():
    rxn = community.merged_model.reactions[rxn_id]
    if rxn_id in maxFiberUTBT:
        community.reactions[rxn_id].lb = -30  
#         rxn.set_flux_bounds(-30, inf) 
    elif rxn_id in maxFiberUTFP:
        community.reactions[rxn_id].lb = -5
#         rxn.set_flux_bounds(-5, inf)
    else:
        rxn.set_flux_bounds(0, 1000)

print(community.merged_model.get_objective())       
        
res = FBA(community.merged_model)
print(res)

# sol = SteadyCom(community)
# print(sol)

{'community_growth': 1}
Objective: None
Status: Unbounded



In [3]:
import cobrakbase
from reframed.core.model import ReactionType

kbase = cobrakbase.KBaseAPI()
ws = 93204
model1 = from_cobrapy(kbase.get_from_ws('Acetobacterium_MES1.electrosynthesis.edited', 93204))
model1.id = 'm1'
model1.compartments.e0.external = True
model2 = from_cobrapy(kbase.get_from_ws('Sulfurospirillum_MES13.autotroph.edited', 93204))
model2.id = 'm2'
model2.compartments.e0.external = True
model3 = from_cobrapy(kbase.get_from_ws('Desulfovibrio_MES4.autotroph.edited.v2', 93204))
model3.id = 'm3'
model3.compartments.e0.external = True

for rxn_id in model1.reactions:
    if 'EX_' in rxn_id:
        rxn = model1.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
for rxn_id in model2.reactions:
    if 'EX_' in rxn_id:
        rxn = model2.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
for rxn_id in model3.reactions:
    if 'EX_' in rxn_id:
        rxn = model3.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
community = Community('electro.community',[model1,model2,model3])

media = kbase.get_from_ws('CO2_minimal', ws) #  we download the media from KBase
media_const = media.get_media_constraints() #  get constraints 
for r in community.merged_model.get_exchange_reactions():
    rxn = community.merged_model.reactions[r]
    if r[5:] in media_const: #  if constraint in media we assign bounds
        lb, ub = media_const[r[5:]]
        rxn.set_flux_bounds(lb, ub)
    else: #  otherwise set to excretion only
        rxn.set_flux_bounds(0, 1000)
# res = FBA(community.merged_model) #  test if new media works

# sol = SteadyCom(community)

In [4]:
# testing model from SteadyCom paper
comModel = load_cbmodel('model9.xml',flavor='fbc2') # flavors: fbc2, cobra, bigg
comModel.id = 'm1'

for comp in comModel.compartments:
    if 'e_' in comp:
        comModel.compartments[comp].external = True
        
for rxn_id in comModel.reactions:
    if '_u_' in rxn_id:
        rxn = comModel.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
        
# included when testing community model in test_reframed.py
for rxn_id in comModel.reactions:
    if 'R_ATPM_' in rxn_id:
        comModel.set_flux_bounds(rxn_id, 0, 0)
    
comm = Community('steadycom_example', [comModel])

maxFiberUTBT = ['R_EX_amannan140_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_pect_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_pecticgal_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_arabinan101_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_arabinogal_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_homogal_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_inulin_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_kesto_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_rhamnogalurII_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_rhamnogalurI_LSQBKT_u_RSQBKT_tr_Bt', \
                'R_EX_kestopt_LSQBKT_u_RSQBKT_tr_Bt', 'R_EX_kestottr_LSQBKT_u_RSQBKT_tr_Bt']

maxFiberUTFP = ['R_IEX_pect_LSQBKT_u_RSQBKT_tr_Fp', 'comModel.reactions.R_IEX_inulin_LSQBKT_u_RSQBKT_tr_Fp', \
                'R_IEX_kesto_LSQBKT_u_RSQBKT_tr_Fp', 'comModel.reactions.R_IEX_kestopt_LSQBKT_u_RSQBKT_tr_Fp', \
                'R_IEX_kestottr_LSQBKT_u_RSQBKT_tr_Fp']

for rxn_id in comm.merged_model.get_exchange_reactions():
    rxn = comm.merged_model.reactions[rxn_id]
    if rxn_id in maxFiberUTBT:
        community.reactions[rxn_id].lb = -30
#         rxn.set_flux_bounds(-30, inf)
    elif rxn_id in maxFiberUTFP:
        community.reactions[rxn_id].lb = -5
#         rxn.set_flux_bounds(-5, inf)
    else:
        rxn.set_flux_bounds(0, 1000)

for rxn_id in comm.merged_model.reactions:
    if rxn_id is not 'community_growth':
        rxn = comm.merged_model.reactions[rxn_id]
        print(rxn_id,rxn.reaction_type) 

env = Environment()
for rxn_id in maxFiberUTBT:
    env[rxn_id] = (-30, inf)
for rxn_id in maxFiberUTFP:
    env[rxn_id] = (-5, inf)

for r_id, rxn in community.merged_model.reactions.items():
    if rxn.objective:
        print(r_id)
        
solFBA = FBA(comModel)
print(solFBA)

solFBA = FBA(comm.merged_model)
print(solFBA)

solSC = SteadyCom(comm)
print(solSC)